In [3]:
import tkinter as tk
from PIL import Image, ImageTk

# Make sure to replace this with the correct path to your baghchal_environment.ipynb file
%run baghchal_environment.ipynb

class BaghchalGUI:
    def __init__(self, master):
        # Initialize the GUI
        self.master = master
        master.title("Baghchal Game")
        
        # Load tiger and goat images (or use simple symbols as a fallback)
        self.tiger_image = Image.new("RGB", (20, 20), color="orange")  # Create a simple orange square for the tiger
        self.goat_image = Image.new("RGB", (20, 20), color="green")  # Create a simple green square for the goat
        
        self.tiger_image = ImageTk.PhotoImage(self.tiger_image)
        self.goat_image = ImageTk.PhotoImage(self.goat_image)
        
        # Create canvas and other GUI elements
        self.canvas = tk.Canvas(master, width=300, height=300)
        self.canvas.pack()
        self.draw_board()

        self.move_label = tk.Label(master, text="")
        self.move_label.pack()

        self.canvas.bind("<Button-1>", self.handle_click)  # Bind left mouse click event

        # Initialize the baghchal board
        self.baghchal = BaghchalBoard()
        self.update_board()
        self.twenty_goat = False 

    def handle_click(self, event):
        # Calculate grid position based on click coordinates
        row = event.y // 60
        col = event.x // 60
        
        # Make move at the clicked position
        self.make_move(row, col)
   
    def draw_board(self):
        # Define the dimensions of the canvas
        canvas_width = 420  # Increased canvas width
        canvas_height = 420  # Increased canvas height
        cell_size = 80  # Size of each grid cell
    
        # Set canvas dimensions
        self.canvas.config(width=canvas_width, height=canvas_height)
        
        # Draw lines to represent connections between points
        for i in range(5):
            for j in range(5):
                x = j * cell_size + cell_size // 2
                y = i * cell_size + cell_size // 2
                if i > 0:
                    self.canvas.create_line(x, y, x, y + 4 * cell_size)  # Extend line to cover the entire canvas height
                if j > 0:
                    self.canvas.create_line(x, y, x + 4 * cell_size, y)  # Extend line to cover the entire canvas width
                if i > 0 and j > 0 and (i + j) % 2 == 0:
                    self.canvas.create_line(x, y, x + 4 * cell_size, y + 4 * cell_size)  # Extend line to cover the entire canvas
                if i > 0 and j < 4 and (i + j) % 2 == 0:
                    self.canvas.create_line(x, y, x + 4 * cell_size, y - 4 * cell_size)  # Extend line to cover the entire canvas
    
    def make_move(self, x, y):
        if self.baghchal.turn == 'G' and self.baghchal.goats_placed < 20:
            # Check if the clicked position is empty ('.')
            if self.baghchal.board[x][y] == '.':
                self.baghchal.board[x][y] = 'G'  # Place the goat
                self.baghchal.goats_placed += 1  # Increment goat count
                self.baghchal.turn = 'T'  # Change to tiger's turn
                self.move_label.config(text=f"Goat placed at ({x}, {y}), Tiger's turn ")
                self.update_board()  # Update the board after placing the goat
            else:
                self.move_label.config(text="Invalid position. Try again.")
        elif self.baghchal.turn == 'G':
            if not self.twenty_goat:
                self.move_label.config(text="Move goat. 20 placements complete ")
            self.twenty_goat = True

            if self.baghchal.board[x][y] == 'G':
                if self.baghchal.selected_goat is None:
                    self.baghchal.selected_goat = (x, y)
                    self.move_label.config(text=f"Selected goat at ({x}, {y}). Click where to move.")
            elif self.baghchal.board[x][y] == '.' and self.baghchal.selected_goat is not None:
                start_x, start_y = self.baghchal.selected_goat
                if self.baghchal.is_valid_move(start_x, start_y, x, y):
                    self.baghchal.make_move(start_x, start_y, x, y)
                    self.baghchal.selected_goat = None
                    self.update_board()
                    self.move_label.config(text="Tigers turn ")
                else:
                    self.move_label.config(text="Invalid move. Try again")
                    self.baghchal.selected_goat = None
            else:
                self.move_label.config(text="Select a goat to move.")
    
        elif self.baghchal.turn == 'T':
            if self.baghchal.board[x][y] == 'T':
                if self.baghchal.selected_tiger is None:
                    self.baghchal.selected_tiger = (x, y)
                    self.move_label.config(text=f"Selected tiger at ({x}, {y}). Click where to move.")
            elif self.baghchal.board[x][y] == '.' and self.baghchal.selected_tiger is not None:
                start_x, start_y = self.baghchal.selected_tiger
                if self.baghchal.is_valid_move(start_x, start_y, x, y):
                    self.baghchal.make_move(start_x, start_y, x, y)
                    self.baghchal.selected_tiger = None
                    self.update_board()
                    self.move_label.config(text='Goat Turn')
                else:
                    self.move_label.config(text="Invalid move. Try again.")
                    self.baghchal.selected_tiger = None
            else:
                self.move_label.config(text="Select a tiger to move.")
        
        result = self.baghchal.check_win_conditions()
        if result is not None:
            if result == 'Goat':
                self.move_label.config(text="Goat wins!!")
                self.disable_board()
            elif result == 'Tiger':
                self.disable_board()
                self.move_label.config(text="Tiger wins!!")

    def disable_board(self):
        # Create a transparent rectangle covering the entire canvas
        self.canvas.create_rectangle(0, 0, 300, 300, fill="", outline="", tags="disable_overlay")
        
        # Bind the rectangle to a dummy function that does nothing
        self.canvas.tag_bind("disable_overlay", "<Button-1>", lambda event: None)
        self.canvas.tag_bind("disable_overlay", "<Button-2>", lambda event: None)
        self.canvas.tag_bind("disable_overlay", "<Button-3>", lambda event: None)
    
    def update_board(self):
        self.canvas.delete("all")  # Clear the canvas
    
        # Draw lines to represent connections between points
        for i in range(5):
            for j in range(5):
                x = j * 60 + 30
                y = i * 60 + 30
                if i > 0:
                    self.canvas.create_line(x, y, x, y - 60)
                if j > 0:
                    self.canvas.create_line(x, y, x - 60, y)
                if i > 0 and j > 0 and (i + j) % 2 == 0:
                    self.canvas.create_line(x, y, x - 60, y - 60)
                if i > 0 and j < 4 and (i + j) % 2 == 0:
                    self.canvas.create_line(x, y, x + 60, y - 60)
    
        # Draw goats and tigers on the canvas
        for i in range(5):
            for j in range(5):
                if self.baghchal.board[i][j] == 'T':
                    x = j * 60 + 30
                    y = i * 60 + 30
                    self.canvas.create_image(x, y, image=self.tiger_image, anchor=tk.CENTER)
                elif self.baghchal.board[i][j] == 'G':
                    x = j * 60 + 30
                    y = i * 60 + 30
                    self.canvas.create_image(x, y, image=self.goat_image, anchor=tk.CENTER)
    
        self.master.update()  # Update the GUI to reflect the changes

def main():
    root = tk.Tk()
    gui = BaghchalGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()
